# Import Libraries

In [94]:
import warnings
warnings.filterwarnings('ignore')

In [138]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import LabelEncoder

from sqlalchemy import create_engine
import psycopg2

In [139]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


# Read the Data from Postgres

In [140]:
#Create a connection to the Postgres & AWS

# Import API key
from config import db_password

connection = psycopg2.connect(
    host = 'finalproject-ml.cvm9etk63tz8.us-west-1.rds.amazonaws.com',
    port = 5432,
    user = 'postgres',
    password = db_password,
    database = 'postgres'
    )
cursor=connection.cursor()

In [141]:
#Read in Table for co_2015
aqi_all = "select * from aqi_all"

In [142]:
#Check DataFrame
aqi_all = pd.read_sql(aqi_all, con=connection)
aqi_all.head()

,Date,aqi_co,aqi_no2,aqi_o3,aqi_pm_2_5,aqi_pm_10
0,2015-01-01,7.0,37.0,26.0,55.0,NaN
1,2015-01-02,8.0,42.0,20.0,73.0,NaN
2,2015-01-03,10.0,45.0,17.0,84.0,NaN
3,2015-01-04,10.0,40.0,19.0,86.0,NaN
4,2015-01-05,9.0,42.0,12.0,69.0,NaN


In [143]:
df = aqi_all.drop(columns=["Date"])

In [144]:
df["AQI_Max"] = pd.DataFrame(df.max(axis=1))

In [145]:
df.head()

,aqi_co,aqi_no2,aqi_o3,aqi_pm_2_5,aqi_pm_10,AQI_Max
0,7.0,37.0,26.0,55.0,NaN,55.0
1,8.0,42.0,20.0,73.0,NaN,73.0
2,10.0,45.0,17.0,84.0,NaN,84.0
3,10.0,40.0,19.0,86.0,NaN,86.0
4,9.0,42.0,12.0,69.0,NaN,69.0


In [146]:
#Handle missing values
df.isnull().sum()

aqi_co          28
aqi_no2         30
aqi_o3          28
aqi_pm_2_5      51
aqi_pm_10     2011
AQI_Max          1
dtype: int64

In [147]:
# Replace missing values with the mean of the column
df_column = ['aqi_co', 'aqi_no2', 'aqi_o3', 'aqi_pm_2_5', 'aqi_pm_10', 'AQI_Max']

for i in df_column:
    a = df[i].mean()
    df[i].replace(np.nan , a,inplace = True)

In [148]:
df.isnull().sum()

aqi_co        0
aqi_no2       0
aqi_o3        0
aqi_pm_2_5    0
aqi_pm_10     0
AQI_Max       0
dtype: int64

In [149]:
df.dtypes

aqi_co        float64
aqi_no2       float64
aqi_o3        float64
aqi_pm_2_5    float64
aqi_pm_10     float64
AQI_Max       float64
dtype: object

In [150]:
df.shape

(2192, 6)

In [151]:
columns = ["aqi_co", "aqi_no2", "aqi_o3", "aqi_pm_2_5", "aqi_pm_10"]

target = ["AQI_Max"]

In [157]:
# Convert the target column values to safe and unsafe based on their values

for aqi in df['AQI_Max']:
    if (aqi > float(50)):
        df['AQI_Max']=df['AQI_Max'].replace(aqi, "Unsafe")
    else:
        df['AQI_Max']=df['AQI_Max'].replace(aqi,"Safe")

In [159]:
df.head()

,aqi_co,aqi_no2,aqi_o3,aqi_pm_2_5,aqi_pm_10,AQI_Max
0,7.0,37.0,26.0,55.0,16.812155,Unsafe
1,8.0,42.0,20.0,73.0,16.812155,Unsafe
2,10.0,45.0,17.0,84.0,16.812155,Unsafe
3,10.0,40.0,19.0,86.0,16.812155,Unsafe
4,9.0,42.0,12.0,69.0,16.812155,Unsafe


In [165]:
df1 = df.groupby(['AQI_Max']).count()

In [166]:
df1

,aqi_co,aqi_no2,aqi_o3,aqi_pm_2_5,aqi_pm_10
AQI_Max,,,,,
Safe,1755,1755,1755,1755,1755
Unsafe,437,437,437,437,437


In [154]:
df.loc[:, 'AQI_Max']

0       55.0
1       73.0
2       84.0
3       86.0
4       69.0
        ... 
2187    28.0
2188    28.0
2189    35.0
2190    32.0
2191    33.0
Name: AQI_Max, Length: 2192, dtype: float64

In [155]:
df.loc[df['AQI_Max'] > float(50)]

,aqi_co,aqi_no2,aqi_o3,aqi_pm_2_5,aqi_pm_10,AQI_Max
0,7.0,37.0,26.0,55.0,16.812155,55.0
1,8.0,42.0,20.0,73.0,16.812155,73.0
2,10.0,45.0,17.0,84.0,16.812155,84.0
3,10.0,40.0,19.0,86.0,16.812155,86.0
4,9.0,42.0,12.0,69.0,16.812155,69.0
...,...,...,...,...,...,...
2179,9.0,29.0,22.0,57.0,16.812155,57.0
2180,9.0,22.0,22.0,55.0,16.812155,55.0
2181,7.0,26.0,20.0,60.0,16.812155,60.0
2184,6.0,29.0,17.0,68.0,16.812155,68.0


In [135]:
df.head()

,aqi_co,aqi_no2,aqi_o3,aqi_pm_2_5,aqi_pm_10,AQI_Max
0,Unsafe,Unsafe,Unsafe,Unsafe,Unsafe,Unsafe
1,Unsafe,Unsafe,Unsafe,Unsafe,Unsafe,Unsafe
2,Unsafe,Unsafe,Unsafe,Unsafe,Unsafe,Unsafe
3,Unsafe,Unsafe,Unsafe,Unsafe,Unsafe,Unsafe
4,Unsafe,Unsafe,Unsafe,Unsafe,Unsafe,Unsafe


In [113]:
df.head()

,aqi_co,aqi_no2,aqi_o3,aqi_pm_2_5,aqi_pm_10,AQI_Max
0,7.0,37.0,26.0,55.0,16.812155,55.0
1,8.0,42.0,20.0,73.0,16.812155,73.0
2,10.0,45.0,17.0,84.0,16.812155,84.0
3,10.0,40.0,19.0,86.0,16.812155,86.0
4,9.0,42.0,12.0,69.0,16.812155,69.0


# Split the Data into Training and Testing

In [69]:
# Create our features
X = df.drop(columns='AQI_Max')

#clean features
X = pd.get_dummies(X)

# Create our target
y = df.loc[:, target].copy()
X.head()

,aqi_co,aqi_no2,aqi_o3,aqi_pm_2_5,aqi_pm_10
0,7.0,37.0,26.0,55.0,16.812155
1,8.0,42.0,20.0,73.0,16.812155
2,10.0,45.0,17.0,84.0,16.812155
3,10.0,40.0,19.0,86.0,16.812155
4,9.0,42.0,12.0,69.0,16.812155


In [70]:
X.describe()

,aqi_co,aqi_no2,aqi_o3,aqi_pm_2_5,aqi_pm_10
count,2192.000000,2192.000000,2192.000000,2192.000000,2192.000000
mean,5.203789,19.962535,27.583641,34.283512,16.812155
std,2.598768,12.791413,8.025458,22.583411,2.987393
min,0.000000,0.000000,3.000000,0.000000,2.000000
25%,3.000000,8.000000,22.000000,20.000000,16.812155
50%,5.000000,18.000000,28.000000,31.000000,16.812155
75%,7.000000,29.000000,32.000000,45.000000,16.812155
max,18.000000,71.000000,108.000000,228.000000,74.000000


In [71]:
# Check the balance of our target values
y["AQI_Max"].value_counts()

31.0     111
36.0      91
33.0      89
30.0      83
35.0      81
        ... 
108.0      1
11.0       1
3.0        1
128.0      1
96.0       1
Name: AQI_Max, Length: 112, dtype: int64

In [72]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [73]:
# Resample the training data with the RandomOversampler

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'AQI_Max': 1})

In [74]:
# Train the Logistic Regression model using the resampled data

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [75]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
print(balanced_accuracy_score(y_test, y_pred))

ValueError: Classification metrics can't handle a mix of continuous and multiclass targets

In [40]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[  75,   26],
       [7528, 9576]])

In [41]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.74      0.56      0.02      0.64      0.42       101
   low_risk       1.00      0.56      0.74      0.72      0.64      0.41     17104

avg / total       0.99      0.56      0.74      0.71      0.64      0.41     17205



### SMOTE Oversampling

In [42]:
# Resample the training data with SMOTE

from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1,sampling_strategy='auto').\
                            fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'loan_status': 1})

In [43]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [44]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
print(balanced_accuracy_score(y_test, y_pred))

0.6548711319915902


In [47]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   63,    38],
       [ 5371, 11733]])

In [48]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.62      0.69      0.02      0.65      0.43       101
   low_risk       1.00      0.69      0.62      0.81      0.65      0.43     17104

avg / total       0.99      0.69      0.62      0.81      0.65      0.43     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [49]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete

from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'loan_status': 1})

In [50]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [51]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
print(balanced_accuracy_score(y_test, y_pred))

0.5442369453268994


In [52]:
confusion_matrix(y_test, y_pred)

array([[   70,    31],
       [10341,  6763]])

In [53]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.69      0.40      0.01      0.52      0.28       101
   low_risk       1.00      0.40      0.69      0.57      0.52      0.27     17104

avg / total       0.99      0.40      0.69      0.56      0.52      0.27     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [54]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete

from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({'loan_status': 1})

In [55]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [56]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
print(balanced_accuracy_score(y_test, y_pred))

0.6483657924959942


In [57]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  73,   28],
       [7287, 9817]])

In [58]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.72      0.57      0.02      0.64      0.42       101
   low_risk       1.00      0.57      0.72      0.73      0.64      0.41     17104

avg / total       0.99      0.57      0.72      0.72      0.64      0.41     17205



## Ensemble Learners

### Balanced Random Forest Classifier

In [ ]:
# Resample the training data with the BalancedRandomForestClassifier
# Create a random forest classifier.
rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

# Fitting the model
rf_model.fit(X_train, y_train)

In [ ]:
# Calculated the balanced accuracy score
predictions = rf_model.predict(X_test)
acc_score = balanced_accuracy_score(y_test, predictions)
acc_score

In [ ]:
# Display the confusion matrix

# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

In [ ]:
# List the features sorted in descending order by feature importance

# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
#importances


# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

### Easy Ensemble AdaBoost Classifier

In [ ]:
# Train the EasyEnsembleClassifier
ee_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)

# Fitting the model
ee_model.fit(X_train, y_train)

In [ ]:
# Calculated the balanced accuracy score
y_pred = ee_model.predict(X_test)
print(balanced_accuracy_score(y_test, y_pred))

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
Summary_AQI_Max_Prediction = pd.DataFrame(
    {"Naive Random Oversampling":{'AccuracyScore': '65%', 'Precision': 0.01, "Recall":0.74,"F1-Score":0.02},
      "SMOTE Oversampling":{'AccuracyScore': '65%', 'Precision': 0.01, "Recall":0.62,"F1-Score":0.02},
     "ClusterCentroids":{'AccuracyScore': '54%', 'Precision': 0.01, "Recall":0.69,"F1-Score":0.01},
     "SMOTEEN":{'AccuracyScore': '65%', 'Precision': 0.01, "Recall":0.72,"F1-Score":0.02},
     "Random Forest Classifier":{'AccuracyScore': '79%', 'Precision': 0.03, "Recall":0.7,"F1-Score":0.06},
     "Easy Ensemble Classifier -AdaBoost":{'AccuracyScore': '93%', 'Precision': 0.09, "Recall":0.92,"F1-Score":0.16}
     })
    
Summary_AQI_Max_Prediction